In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from datetime import datetime
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Passo 1 - Importar os dados

In [8]:
#import data
Products=pd.read_excel("data/Products.xlsx")
Products.set_index('Cust_ID', inplace = True)
Products.head()

,Fitness&Gym,Hiking&Running,Last_Purchase,TeamGames,OutdoorActivities,TotalProducts
Cust_ID,,,,,,
4,0,76,2024-02-19,248,37,6
5,0,75,2024-02-17,248,37,6
6,6,69,2024-02-15,215,34,5
9,36,71,2024-02-13,248,35,8
10,8,97,2024-02-15,204,34,5


In [9]:
#search duplicates
len(Products[Products.duplicated()])

510

There are 510 duplicates.  

However, there is no reason to exclude them when the feedback we get is that the data's integrity is of the highst quality and all IDs (common with other two datasets are unique. It could be a specific promotion, for example.

In [13]:
#confirm uniqueness of IDs
len(Products.index.unique())

4000

In [19]:
#make column names better with snake_case
Products.columns = Products.columns.str.lower().str.replace('&','_')
Products.columns

Index(['fitness_gym', 'hiking_running', 'last_purchase', 'teamgames',
       'outdooractivities', 'totalproducts'],
      dtype='object')

# Visualização de Característias básicas dos DF

In [20]:
Products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, 4 to 6496
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fitness_gym        4000 non-null   int64         
 1   hiking_running     4000 non-null   int64         
 2   last_purchase      4000 non-null   datetime64[ns]
 3   teamgames          4000 non-null   int64         
 4   outdooractivities  4000 non-null   int64         
 5   totalproducts      4000 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 347.8 KB


All value types seem correct. No null values.

In [21]:
Products.describe()

,fitness_gym,hiking_running,last_purchase,teamgames,outdooractivities,totalproducts
count,4000.000000,4000.000000,4000,4000.000000,4000.000000,4000.000000
mean,32.020250,55.844750,2024-01-30 18:50:02.399999744,176.085000,33.602750,5.329500
min,0.000000,9.000000,2023-10-15 00:00:00,0.000000,28.000000,2.000000
25%,25.000000,38.000000,2024-01-20 00:00:00,121.000000,32.000000,4.000000
50%,31.000000,47.000000,2024-02-01 00:00:00,180.000000,33.000000,5.000000
75%,40.000000,66.000000,2024-02-13 00:00:00,230.000000,35.000000,6.000000
max,123.000000,464.000000,2024-02-29 00:00:00,1203.000000,42.000000,20.000000
std,14.469284,33.119109,NaN,69.964738,1.711483,1.515257


All features have median and mean values close together, suggesting gaussian distribution data.  

Let's confirm.

In [22]:
#check skewness of data
numeric=['fitness_gym', 'hiking_running', 'teamgames', 'outdooractivities', 'totalproducts']
Products[numeric].skew()

fitness_gym          0.346342
hiking_running       4.671687
teamgames            0.740859
outdooractivities    0.414896
totalproducts        1.791119
dtype: float64

hiking_running and totalproducts  are highly positively skewed.  
teamgames is moderately skewed.


In [23]:
Products[numeric].kurt()

fitness_gym           1.464810
hiking_running       37.839539
teamgames            10.605494
outdooractivities     0.399144
totalproducts         9.247286
dtype: float64

In [26]:
#check distributions
sns.boxplot(data=Products[numeric], x=Products[numeric].columns, y=Products[numeric])

ValueError: Length of Index vectors must match length of `data` when both are used, but `data` has length 4000 and the vector passed to `x` has length 5.

# Passo 2- Pré-processadamente do DF

No Df Products, criamos
1) Coluna de Total de Gasto,
2) Gasto médio por produto e
3) Trocamos data de última compra por #dias desde da última compra (assumindo 21-04-2024)
4) Removemos os duplicados
5) Deixar cair a coluna 'TotalProducts' porque não dá grande informação.

In [29]:
# Criar a nova coluna "Total_Expensed" com a soma das colunas especificadas
spending=['fitness_gym', 'hiking_running', 'teamgames', 'outdooractivities']
Products['Total_Expensed'] = Products[spending].sum(axis=1)
Products.head()                                                   

,fitness_gym,hiking_running,last_purchase,teamgames,outdooractivities,totalproducts,Total_Expensed
Cust_ID,,,,,,,
4,0,76,2024-02-19,248,37,6,361
5,0,75,2024-02-17,248,37,6,360
6,6,69,2024-02-15,215,34,5,324
9,36,71,2024-02-13,248,35,8,390
10,8,97,2024-02-15,204,34,5,343


In [ ]:
Products['Avg_Spent']=Products['Total_Expensed']/Products['TotalProducts']
Products['Avg_Spent'] = np.ceil(Products['Avg_Spent'])  # Arredondar para cima
Products['Avg_Spent']=Products['Avg_Spent'].astype(int)
Products